In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

from helper.helper_functions import load_dataset, save_model, get_features_and_target, encode_all_features, get_train_test_with_excluded_columns
from helper.clfmodel_functions import tune_model, forward_feat_selection_hypertuning

from helper.fairness_functions import print_male_female_metrics, get_male_female_data

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.inspection import permutation_importance

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'
pip install 'aif360[OptimalTransport]'


### Loading the cleaned dataset

In [2]:
data: pd.DataFrame = load_dataset('../data/assignment2_income_cleaned.xlsx')

### Feature Engineering (encoding) & Train-Test Split

In [3]:
# Splitting the data into features (X) and target (y)
X, y = get_features_and_target(data, 'income')
X_male, X_female = get_male_female_data(X, False)
# Encoding the features and target
X_encoded, y_encoded = encode_all_features(X, y, [])
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

In [4]:
X_train

,age,education,workinghours,ability to speak english,occupation_Construction/Extraction,occupation_Counseling/Mental Health Services,occupation_Education,occupation_Entertainment,"occupation_Farming, Fishing, Forestry",occupation_Finance/Accounting,...,marital status_Widowed,marital status_Wife,gave birth this year_No,gave birth this year_Yes,sex_Female,sex_Male,workclass_governmental,workclass_no paid work,workclass_private,workclass_self employed
6317,22,16,36,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
740,61,22,40,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
3781,48,16,40,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
7850,62,18,65,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
2963,53,19,44,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,22,19,25,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
5191,24,16,28,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
5390,35,16,40,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
860,23,20,40,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0


### Model

Here, we quickly train and evaluate a Logistic Regression model with random parameters for demonstration.

In [5]:
# Logistic Regression model
lr_model = LogisticRegression(max_iter=2000)
lr_model.fit(X_train, y_train)
lr_preds = lr_model.predict(X_test)
lr_accuracy = accuracy_score(y_test, lr_preds)

print(classification_report(y_test, lr_preds))
print("Logistic Regression Accuracy:", lr_accuracy)

              precision    recall  f1-score   support

           0       0.82      0.87      0.85      1175
           1       0.73      0.64      0.68       625

    accuracy                           0.79      1800
   macro avg       0.77      0.76      0.76      1800
weighted avg       0.79      0.79      0.79      1800

Logistic Regression Accuracy: 0.7922222222222223


### Feature Importance using the model itself

In [6]:
# Get the coefficients of the logistic regression model
feature_importance_scores = lr_model.coef_[0]
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': feature_importance_scores
})
# We sort the DataFrame by the absolute value of the coefficients
feature_importance_df['Absolute_Coefficient'] = feature_importance_df['Coefficient'].abs()
feature_importance_df = feature_importance_df.sort_values(by='Absolute_Coefficient', ascending=False).drop(columns='Absolute_Coefficient')

In [7]:
feature_importance_df

,Feature,Coefficient
20,occupation_Service/Hospitality,-1.549526
19,"occupation_Science, Engineering, Technology",1.219059
9,occupation_Finance/Accounting,0.966728
11,occupation_Legal Services,0.812925
12,occupation_Management/Business,0.790683
8,"occupation_Farming, Fishing, Forestry",-0.648786
24,marital status_Never married,-0.621236
33,workclass_no paid work,-0.587741
13,occupation_Military Services,0.577947
23,marital status_Husband,0.571337


In [8]:
# https://stackoverflow.com/questions/34052115/how-to-find-the-importance-of-the-features-for-a-logistic-regression-model
# https://scikit-learn.org/stable/modules/permutation_importance.html
model_fi = permutation_importance(lr_model, X_encoded, y_encoded, n_repeats=30, random_state=0)
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': model_fi['importances_mean']
})
feature_importance_df = feature_importance_df.sort_values(by='Coefficient', ascending=False)

In [9]:
feature_importance_df

,Feature,Coefficient
1,education,0.036693
2,workinghours,0.035622
0,age,0.028467
20,occupation_Service/Hospitality,0.010600
23,marital status_Husband,0.007711
12,occupation_Management/Business,0.007293
19,"occupation_Science, Engineering, Technology",0.007252
24,marital status_Never married,0.005015
32,workclass_governmental,0.003307
15,occupation_Production/Assembly,0.002093


### Hyperparameter Tuning

Here, we define the base parameter grid for our hyperparameter tuning function.

In [10]:
param_grid = {
        'penalty': ['l2'],
        'C': [0.01, 0.1, 1, 10],
        'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky'],
        'max_iter': [5000],
        'random_state': [42]
    }

Below, we tune the hyperparameters of the Logistic Regression model using the defined parameter grid and using all features.

In [11]:
best_params, best_model, best_accuracy = tune_model(LogisticRegression(), X_encoded, X_train, y_train, X_test, y_test, param_grid)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

lr_preds = best_model.predict(X_test)
print(classification_report(y_test, lr_preds))

print_male_female_metrics(best_model, X, X_male, X_female, X_test, y_test)
save_model(best_model, '../output/saved_models/lr_model_sexincl_noffs_tuned_unfair.joblib')

Fitting 5 folds for each of 16 candidates, totalling 80 fits

Best Hyperparameters: {'C': 1, 'max_iter': 5000, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs'}
Best Model: LogisticRegression(C=1, max_iter=5000, random_state=42)
Best Model Accuracy: 0.7922222222222223
              precision    recall  f1-score   support

           0       0.82      0.87      0.85      1175
           1       0.73      0.64      0.68       625

    accuracy                           0.79      1800
   macro avg       0.77      0.76      0.76      1800
weighted avg       0.79      0.79      0.79      1800

Fairness Metrics:
Male FPR: 0.1777150916784203
Male TPR: 0.7314629258517034
Female FPR: 0.0536480686695279
Female TPR: 0.29365079365079366

Disparate Impact (DI): 0.258
Discrimination Score (DS): -0.302
Equal Opportunity Difference (EO): 0.438
Equalized Odds (EOdds): 0.124


Now the same but with fairness constraints.

In [12]:
best_params, best_model, best_accuracy = tune_model(LogisticRegression(), X_encoded, X_train, y_train, X_test, y_test, param_grid, ensure_fairness=True)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

lr_preds = best_model.predict(X_test)
print(classification_report(y_test, lr_preds))

print_male_female_metrics(best_model, X, X_male, X_female, X_test, y_test)
save_model(best_model, '../output/saved_models/lr_model_sexincl_noffs_tuned_fair.joblib')

Fitting 5 folds for each of 16 candidates, totalling 80 fits

Best Hyperparameters: {'C': 10, 'max_iter': 5000, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs'}
Best Model: LogisticRegression(C=10, max_iter=5000, random_state=42)
Best Model Accuracy: 0.7911111111111111
              precision    recall  f1-score   support

           0       0.82      0.87      0.84      1175
           1       0.72      0.64      0.68       625

    accuracy                           0.79      1800
   macro avg       0.77      0.76      0.76      1800
weighted avg       0.79      0.79      0.79      1800

Fairness Metrics:
Male FPR: 0.17912552891396333
Male TPR: 0.7294589178356713
Female FPR: 0.055793991416309016
Female TPR: 0.30158730158730157

Disparate Impact (DI): 0.266
Discrimination Score (DS): -0.298
Equal Opportunity Difference (EO): 0.428
Equalized Odds (EOdds): 0.123


Below, we tune the hyperparameters of the Logistic Regression model using the defined parameter grid and using a **subset of features**. We exclude the columns 'age', 'ability to speak english', and 'workclass'.

In [13]:
columns_to_exclude = ['age', 'ability to speak english', 'workclass']
X_train_, X_test_, y_train_, y_test_, X_male, X_female = get_train_test_with_excluded_columns(data, columns_to_exclude)

best_params, best_model, best_accuracy = tune_model(LogisticRegression(), X_encoded, X_train_, y_train_, X_test_, y_test_, param_grid)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

lr_preds = best_model.predict(X_test_)
print(classification_report(y_test_, lr_preds))

print_male_female_metrics(best_model, X, X_male, X_female, X_test_, y_test_)
save_model(best_model, '../output/saved_models/lr_model_sexincl_noffs_tuned_unfair_aawexcl.joblib')

Fitting 5 folds for each of 16 candidates, totalling 80 fits

Best Hyperparameters: {'C': 10, 'max_iter': 5000, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs'}
Best Model: LogisticRegression(C=10, max_iter=5000, random_state=42)
Best Model Accuracy: 0.7733333333333333
              precision    recall  f1-score   support

           0       0.80      0.87      0.83      1175
           1       0.71      0.59      0.64       625

    accuracy                           0.77      1800
   macro avg       0.75      0.73      0.74      1800
weighted avg       0.77      0.77      0.77      1800

Fairness Metrics:
Male FPR: 0.18053596614950634
Male TPR: 0.6793587174348698
Female FPR: 0.0536480686695279
Female TPR: 0.24603174603174602

Disparate Impact (DI): 0.245
Discrimination Score (DS): -0.292
Equal Opportunity Difference (EO): 0.433
Equalized Odds (EOdds): 0.127


Now the same but with fairness constraints.

In [14]:
columns_to_exclude = ['age', 'ability to speak english', 'workclass']
X_train_, X_test_, y_train_, y_test_, X_male, X_female = get_train_test_with_excluded_columns(data, columns_to_exclude)

best_params, best_model, best_accuracy = tune_model(LogisticRegression(), X_encoded, X_train_, y_train_, X_test_, y_test_, param_grid, ensure_fairness=True)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

lr_preds = best_model.predict(X_test_)
print(classification_report(y_test_, lr_preds))

print_male_female_metrics(best_model, X, X_male, X_female, X_test_, y_test_)
save_model(best_model, '../output/saved_models/lr_model_sexincl_noffs_tuned_fair_aawexcl.joblib')

Fitting 5 folds for each of 16 candidates, totalling 80 fits

Best Hyperparameters: {'C': 10, 'max_iter': 5000, 'penalty': 'l2', 'random_state': 42, 'solver': 'liblinear'}
Best Model: LogisticRegression(C=10, max_iter=5000, random_state=42, solver='liblinear')
Best Model Accuracy: 0.7722222222222223
              precision    recall  f1-score   support

           0       0.80      0.87      0.83      1175
           1       0.70      0.59      0.64       625

    accuracy                           0.77      1800
   macro avg       0.75      0.73      0.74      1800
weighted avg       0.77      0.77      0.77      1800

Fairness Metrics:
Male FPR: 0.18053596614950634
Male TPR: 0.6793587174348698
Female FPR: 0.05793991416309013
Female TPR: 0.24603174603174602

Disparate Impact (DI): 0.253
Discrimination Score (DS): -0.289
Equal Opportunity Difference (EO): 0.433
Equalized Odds (EOdds): 0.123


#### Forward Feature Selection with Hyperparameter Tuning

Because hyperparameter tuning results can vary depending on the chosen subset of features, we will use forward feature selection with hyperparameter tuning to find the best subset of features and hyperparameters for the model.

In [15]:
# Forward feature selection with hyperparameter tuning
best_subset, best_params, best_score = forward_feat_selection_hypertuning(LogisticRegression(), param_grid, X_encoded, X_train, y_train, X_test, y_test)

print("Best subset of features:", best_subset)
print("Best hyperparameters:", best_params)
print("Best model accuracy:", best_score)

# Use the best subset and best hyperparameters for final model
final_model_allf_ffs = LogisticRegression(**best_params)
final_model_allf_ffs.fit(X_train[best_subset], y_train)
final_model_preds = final_model_allf_ffs.predict(X_test[best_subset])
final_model_accuracy = accuracy_score(y_test, final_model_preds)

print(classification_report(y_test, final_model_preds))

print_male_female_metrics(final_model_allf_ffs, X, X_male, X_female, X_test[best_subset], y_test)
save_model(final_model_allf_ffs, '../output/saved_models/lr_model_sexincl_ffs_tuned_unfair.joblib')

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best subset: ['occupation_Construction/Extraction', 'occupation_Counseling/Mental Health Services', 'occupation_Education', 'occupation_Entertainment', 'occupation_Farming, Fishing, Forestry', 'occupation_Finance/Accounting', 'occupation_Healthcare/Medical Services', 'occupation_Legal Services', 'occupation_Management/Business', 'occupation_Military Services', 'occupation_Office/Administrative Support', 'occupation_Production/Assembly', 'occupatio

Now the same but with fairness constraints.

In [16]:
# Forward feature selection with hyperparameter tuning
best_subset, best_params, best_score = forward_feat_selection_hypertuning(LogisticRegression(), param_grid, X_encoded, X_train, y_train, X_test, y_test, ensure_fairness=True)

print("Best subset of features:", best_subset)
print("Best hyperparameters:", best_params)
print("Best model accuracy:", best_score)

# Use the best subset and best hyperparameters for final model
final_model_allf_ffs = LogisticRegression(**best_params)
final_model_allf_ffs.fit(X_train[best_subset], y_train)
final_model_preds = final_model_allf_ffs.predict(X_test[best_subset])
final_model_accuracy = accuracy_score(y_test, final_model_preds)

print(classification_report(y_test, final_model_preds))

print_male_female_metrics(final_model_allf_ffs, X, X_male, X_female, X_test[best_subset], y_test)
save_model(final_model_allf_ffs, '../output/saved_models/lr_model_sexincl_ffs_tuned_fair.joblib')

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best subset: ['sex_Female', 'sex_Male', 'occupation_Construction/Extraction', 'occupation_Counseling/Mental Health Services', 'occupation_Education', 'occupation_Entertainment', 'occupation_Farming, Fishing, Forestry', 'occupation_Finance/Accounting', 'occupation_Healthcare/Medical Services', 'occupation_Legal Services', 'occupation_Management/Business', 'occupation_Military Services', 'occupation_Office/Administrative Support', 'occupation_Production/Assembly', 'occupation_Protective Services', 'occupation

#### Removing sex-related columns

Below we do Forward feature selection with hyperparameter tuning, but we exclude all sex-related columns.

In [17]:
# Forward feature selection with hyperparameter tuning
columns_to_exclude = ['sex', 'gave birth this year', 'marital status']
X_train_, X_test_, y_train_, y_test_, X_male, X_female = get_train_test_with_excluded_columns(data, columns_to_exclude)

best_subset, best_params, best_score = forward_feat_selection_hypertuning(LogisticRegression(), param_grid, X_encoded, X_train_, y_train_, X_test_, y_test_)

print("Best subset of features:", best_subset)
print("Best hyperparameters:", best_params)
print("Best model accuracy:", best_score)

# Use the best subset and best hyperparameters for final model
final_model_sexexcl_ffs = LogisticRegression(**best_params)
final_model_sexexcl_ffs.fit(X_train_[best_subset], y_train_)
final_model_preds = final_model_sexexcl_ffs.predict(X_test_[best_subset])
final_model_accuracy = accuracy_score(y_test_, final_model_preds)

print(classification_report(y_test, final_model_preds))

print_male_female_metrics(final_model_sexexcl_ffs, X, X_male, X_female, X_test_[best_subset], y_test_)
save_model(final_model_sexexcl_ffs, '../output/saved_models/lr_model_sexexcl_ffs_tuned_unfair.joblib')

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best subset: ['occupation_Construction/Extraction', 'occupation_Counseling/Mental Health Services', 'occupation_Education', 'occupation_Entertainment', 'occupation_Farming, Fishing, Forestry', 'occupation_Finance/Accounting', 'occupation_Healthcare/Medical Services', 'occupation_Legal Services', 'occupation_Management/Business', 'occupation_Military Services', 'occupation_Office/Administrative Support', 'occupation_Production/Assembly', 'occupation_Protective Services', 'occupation_Repair/Maintenance', 'occupation_Sales', 'occupation_Science, Engineering, Technology', 'occupation_Service/Hospitality', 'occupation_Transport']
R

Now the same but with fairness constraints.

In [18]:
# Forward feature selection with hyperparameter tuning
columns_to_exclude = ['sex', 'gave birth this year', 'marital status']
X_train_, X_test_, y_train_, y_test_, X_male, X_female = get_train_test_with_excluded_columns(data, columns_to_exclude)

best_subset, best_params, best_score = forward_feat_selection_hypertuning(LogisticRegression(), param_grid, X_encoded, X_train_, y_train_, X_test_, y_test_, ensure_fairness=True)

print("Best subset of features:", best_subset)
print("Best hyperparameters:", best_params)
print("Best model accuracy:", best_score)

# Use the best subset and best hyperparameters for final model
final_model_sexexcl_ffs = LogisticRegression(**best_params)
final_model_sexexcl_ffs.fit(X_train_[best_subset], y_train_)
final_model_preds = final_model_sexexcl_ffs.predict(X_test_[best_subset])
final_model_accuracy = accuracy_score(y_test_, final_model_preds)

print(classification_report(y_test, final_model_preds))

print_male_female_metrics(final_model_sexexcl_ffs, X, X_male, X_female, X_test_[best_subset], y_test_)
save_model(final_model_sexexcl_ffs, '../output/saved_models/lr_model_sexexcl_ffs_tuned_unfair.joblib')

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best subset: ['occupation_Construction/Extraction', 'occupation_Counseling/Mental Health Services', 'occupation_Education', 'occupation_Entertainment', 'occupation_Farming, Fishing, Forestry', 'occupation_Finance/Accounting', 'occupation_Healthcare/Medical Services', 'occupation_Legal Services', 'occupation_Management/Business', 'occupation_Military Services', 'occupation_Office/Administrative Support', 'occupation_Production/Assembly', 'occupation_Protective Services', 'occupation_Repair/Maintenance', 'occupation_Sales', 'occupation_Science, Engineering, Technology', 'occupation_Service/Hospitality', 'occupation_Transport']
R

### AdaBoost Classifier

In [19]:
# Define the parameter grid for AdaBoost
param_grid = {
    'algorithm': ['SAMME', 'SAMME.R'],
    'n_estimators': [50, 75, 100, 150],
    'learning_rate': [0.1, 0.5, 1.0],
    'estimator': [LogisticRegression(random_state=42), final_model_allf_ffs, final_model_sexexcl_ffs],
    'random_state': [42]
}

Below, we tune the hyperparameters of the AdaBoost model using the defined parameter grid and using all features.

In [20]:
best_params, best_model, best_accuracy = tune_model(AdaBoostClassifier(), X_encoded, X_train, y_train, X_test, y_test, param_grid)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

preds = best_model.predict(X_test)
print(classification_report(y_test, preds))

print_male_female_metrics(best_model, X, X_male, X_female, X_test, y_test)
save_model(best_model, '../output/saved_models/lr_model_sexincl_noffs_tuned_unfair_ada.joblib')

Fitting 5 folds for each of 72 candidates, totalling 360 fits

Best Hyperparameters: {'algorithm': 'SAMME.R', 'estimator': LogisticRegression(C=10, max_iter=5000, random_state=42, solver='newton-cg'), 'learning_rate': 1.0, 'n_estimators': 100, 'random_state': 42}
Best Model: AdaBoostClassifier(estimator=LogisticRegression(C=10, max_iter=5000,
                                                random_state=42,
                                                solver='newton-cg'),
                   n_estimators=100, random_state=42)
Best Model Accuracy: 0.7838888888888889
              precision    recall  f1-score   support

           0       0.81      0.87      0.84      1175
           1       0.71      0.63      0.67       625

    accuracy                           0.78      1800
   macro avg       0.76      0.75      0.75      1800
weighted avg       0.78      0.78      0.78      1800

Fairness Metrics:
Male FPR: 0.18899858956276447
Male TPR: 0.717434869739479
Female FPR: 0.0493562231

Now the same but with fairness constraints.

In [21]:
best_params, best_model, best_accuracy = tune_model(AdaBoostClassifier(), X_encoded, X_train, y_train, X_test, y_test, param_grid, ensure_fairness=True)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

preds = best_model.predict(X_test)
print(classification_report(y_test, preds))

print_male_female_metrics(best_model, X, X_male, X_female, X_test, y_test)
save_model(best_model, '../output/saved_models/lr_model_sexincl_noffs_tuned_fair_ada.joblib')

Fitting 5 folds for each of 72 candidates, totalling 360 fits

Best Hyperparameters: {'algorithm': 'SAMME', 'estimator': LogisticRegression(C=10, max_iter=5000, random_state=42, solver='newton-cg'), 'learning_rate': 0.5, 'n_estimators': 100, 'random_state': 42}
Best Model: AdaBoostClassifier(algorithm='SAMME',
                   estimator=LogisticRegression(C=10, max_iter=5000,
                                                random_state=42,
                                                solver='newton-cg'),
                   learning_rate=0.5, n_estimators=100, random_state=42)
Best Model Accuracy: 0.7661111111111111
              precision    recall  f1-score   support

           0       0.79      0.88      0.83      1175
           1       0.71      0.56      0.62       625

    accuracy                           0.77      1800
   macro avg       0.75      0.72      0.73      1800
weighted avg       0.76      0.77      0.76      1800

Fairness Metrics:
Male FPR: 0.165021156558533

Below, we tune the hyperparameters of the AdaBoost model using the defined parameter grid and using a **subset of features**. We exclude the sex-related columns.

In [22]:
columns_to_exclude = ['sex', 'gave birth this year', 'marital status']
X_train_, X_test_, y_train_, y_test_, X_male, X_female = get_train_test_with_excluded_columns(data, columns_to_exclude)

best_params, best_model, best_accuracy = tune_model(AdaBoostClassifier(), X_encoded, X_train_, y_train_, X_test_, y_test_, param_grid)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

preds = best_model.predict(X_test_)
print(classification_report(y_test_, preds))

print_male_female_metrics(best_model, X, X_male, X_female, X_test_, y_test_)
save_model(best_model, '../output/saved_models/lr_model_sexexcl_noffs_tuned_unfair_ada.joblib')

Fitting 5 folds for each of 72 candidates, totalling 360 fits

Best Hyperparameters: {'algorithm': 'SAMME.R', 'estimator': LogisticRegression(C=10, max_iter=5000, random_state=42, solver='newton-cg'), 'learning_rate': 0.5, 'n_estimators': 75, 'random_state': 42}
Best Model: AdaBoostClassifier(estimator=LogisticRegression(C=10, max_iter=5000,
                                                random_state=42,
                                                solver='newton-cg'),
                   learning_rate=0.5, n_estimators=75, random_state=42)
Best Model Accuracy: 0.7544444444444445
              precision    recall  f1-score   support

           0       0.78      0.86      0.82      1175
           1       0.68      0.55      0.61       625

    accuracy                           0.75      1800
   macro avg       0.73      0.71      0.71      1800
weighted avg       0.75      0.75      0.75      1800

Fairness Metrics:
Male FPR: 0.14809590973201692
Male TPR: 0.5851703406813628
Female

Now the same but with fairness constraints.

In [23]:
columns_to_exclude = ['sex', 'gave birth this year', 'marital status']
X_train_, X_test_, y_train_, y_test_, X_male, X_female = get_train_test_with_excluded_columns(data, columns_to_exclude)

best_params, best_model, best_accuracy = tune_model(AdaBoostClassifier(), X_encoded, X_train_, y_train_, X_test_, y_test_, param_grid, ensure_fairness=True)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

preds = best_model.predict(X_test_)
print(classification_report(y_test_, preds))

print_male_female_metrics(best_model, X, X_male, X_female, X_test_, y_test_)
save_model(best_model, '../output/saved_models/lr_model_sexexcl_noffs_tuned_fair_ada.joblib')

Fitting 5 folds for each of 72 candidates, totalling 360 fits

Best Hyperparameters: {'algorithm': 'SAMME', 'estimator': LogisticRegression(C=10, max_iter=5000, random_state=42, solver='newton-cg'), 'learning_rate': 0.1, 'n_estimators': 75, 'random_state': 42}
Best Model: AdaBoostClassifier(algorithm='SAMME',
                   estimator=LogisticRegression(C=10, max_iter=5000,
                                                random_state=42,
                                                solver='newton-cg'),
                   learning_rate=0.1, n_estimators=75, random_state=42)
Best Model Accuracy: 0.7511111111111111
              precision    recall  f1-score   support

           0       0.77      0.87      0.82      1175
           1       0.69      0.52      0.59       625

    accuracy                           0.75      1800
   macro avg       0.73      0.70      0.71      1800
weighted avg       0.74      0.75      0.74      1800

Fairness Metrics:
Male FPR: 0.11283497884344147